In [1]:
!nvidia-smi

Thu Jun 27 09:20:19 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:86:00.0 Off |                  N/A |
| 59%   71C    P2   236W / 350W |  16870MiB / 24268MiB |     10%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install tensorflow

     |████████████████████████████████| 588.3 MB 6.9 kB/s eta 0:00:01    |█▊                              | 32.2 MB 289 kB/s eta 0:32:01     |██                              | 36.4 MB 584 kB/s eta 0:15:45     |██████▉                         | 125.2 MB 603 kB/s eta 0:12:48     |███████████████▍                | 283.1 MB 719 kB/s eta 0:07:05     |███████████████▌                | 284.5 MB 724 kB/s eta 0:07:00     |████████████████                | 295.6 MB 738 kB/s eta 0:06:37     |██████████████████████████▊     | 491.4 MB 392 kB/s eta 0:04:07     |██████████████████████████▊     | 491.4 MB 392 kB/s eta 0:04:07     |████████████████████████████▎   | 518.9 MB 276 kB/s eta 0:04:11
     |████████████████████████████████| 439 kB 370 kB/s eta 0:00:01
     |████████████████████████████████| 65 kB 446 kB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 277 kB/s eta 0:00:01
     |████████████████████████████████| 133 kB 320 kB/s eta 0:00:01
     |████████████████████████████████| 4

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, MultiHeadAttention, Reshape, GlobalAveragePooling2D,Add
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
from tensorflow.keras.callbacks import EarlyStopping
import os
import cv2  # You may need to install opencv-python using `pip install opencv-python`
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import normalize


2024-06-27 10:00:16.781326: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-27 10:00:16.946579: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-06-27 10:00:16.946609: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-06-27 10:00:17.838531: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinf

ModuleNotFoundError: No module named 'cv2'

In [5]:
def load_image(image_path, target_size=(142, 142)):
    """Load an image from file and resize it to the target size."""
    img = load_img(image_path, target_size=target_size,color_mode='grayscale'	)
    img_array = img_to_array(img)
    img_array = img_array / img_array.max() # Normalize to [0, 1]
    # img_array = (img_array > 0.5).astype(np.float32)

    return img_array

# Constants
main_folder = "./processed"  # Replace with your main folder path
target_image_size = (142, 142)  # Size to resize the images

def load_data(folder_path):
    target_images = []
    X2_train_images = []
    y1_train = []
    y2_train = []

    for subdir, dirs, files in os.walk(folder_path):
        print(subdir,dirs)
        for file in files:
            if file == 'image_data.csv':
                csv_path = os.path.join(subdir, file)
                df = pd.read_csv(csv_path)
                target_image_path = os.path.join(subdir, 'target.jpg')
                target_img = load_img(target_image_path, target_size=target_image_size)
                target_img_array = img_to_array(target_img)
                target_img_array = normalize(target_img_array)
                target_images.append(target_img_array)

                for index, row in df.iterrows():
                    # print(subdir)
                    # print(csv_path)

                    X2_image_path = os.path.join(subdir, row['Image_Name'])
                    X2_img = load_img(X2_image_path, target_size=target_image_size)
                    X2_img_array = img_to_array(X2_img)
                    X2_img_array = normalize(X2_img_array)
                    X2_train_images.append(X2_img_array)
                    y1_train.append(row['x'])
                    y2_train.append(row['y'])
                    # print("ggg")

    target_images = np.array(target_images)
    X2_train_images = np.array(X2_train_images)
    y1_train = np.array(y1_train)
    y2_train = np.array(y2_train)

    return target_images, X2_train_images, y1_train, y2_train

# Example usage
target_images, X2_train_images, y1_train, y2_train = load_data(main_folder)

print("Shape of target_images:", target_images.shape)
print("Shape of X2_train_images:", X2_train_images.shape)
print("Shape of y1_train:", y1_train.shape)
print("Shape of y2_train:", y2_train.shape)



./processed ['D8_1_2_3', 'D7_1_2', 'D8_1_2', 'D1', 'D8', 'D9_1_2', 'D9_1', 'D3_1', 'D4_1_2_3', 'D4_1', 'D4_1_2', 'D6_1_2', 'D3_1_2_3', 'D2_1_2', 'D4', 'D5_1_2', 'D9', 'D5_1_2_3', 'D3_2', 'D3', 'D2', 'D2_1', 'D8_1', 'D6_1', 'D5_1', 'D1_1', 'D7_1', 'D5', 'D1_1_2_3', 'D2_1_2_3', 'D3_1_2', 'D7', 'D9_1_2_3', 'D6', 'D7_1_2_3', 'D1_1_2', 'D6_1_2_3']
./processed/D8_1_2_3 []


NameError: name 'normalize' is not defined

In [17]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  0


In [5]:
filter=8


def apply_attention(x, num_filters):
    # Reshape for attention layer
    b, h, w, c = x.shape
    x_reshaped = Reshape((h * w, c))(x)
    
    # Apply Multi-Head Attention
    attention = MultiHeadAttention(num_heads=4, key_dim=num_filters)(x_reshaped, x_reshaped)
    
    # Reshape back to original dimensions
    attention = Reshape((h, w, c))(attention)
    
    # Add residual connection
    x = Add()([x, attention])
    
    return x


def create_cnn_branch(input_shape):
    inputs = Input(shape=input_shape)
    attentionn = MultiHeadAttention(num_heads=4, key_dim=32)

    x = Conv2D(8, (3, 3),padding='same', activation='relu')(inputs)
    x = Conv2D(8, (3, 3),padding='same', activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = apply_attention(x, filter)

    x = Conv2D(16, (3, 3),padding='same', activation='relu')(x)
    x = Conv2D(16, (3, 3),padding='same', activation='relu')(x)
    x= attentionn(x,x)
    x = MaxPooling2D((2, 2))(x)
    x = apply_attention(x, filter+8)

    x = Conv2D(24, (3, 3), padding='same',activation='relu')(x)
    x = Conv2D(24, (3, 3), padding='same',activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = apply_attention(x, filter+16)


    # x = Flatten()(x)
    return inputs, x

def create_model(input_shape):
    input1, branch1 = create_cnn_branch(input_shape)
    input2, branch2 = create_cnn_branch(input_shape)
    # x= attention(input1,input2)

    attention = MultiHeadAttention(num_heads=4, key_dim=28)
    x = attention(branch1, branch2)

    x = Conv2D(32, (3, 3),padding='same', activation='relu')(x)
    x = Conv2D(32, (3, 3),padding='same', activation='relu')(x)
    # x= attention(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(40, (3, 3),padding='same', activation='relu')(x)
    x = Conv2D(40, (3, 3),padding='same', activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(48, (3, 3), padding='same',activation='relu')(x)
    x = Conv2D(48, (3, 3), padding='same',activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)

    x = Dense(64, activation='relu')(x)
    output1 = Dense(1, activation='tanh')(x)
    output2 = Dense(1 , activation='tanh')(x)

    model = Model(inputs=[input1, input2], outputs=[output1, output2])
    return model


In [6]:
input_shape = (142, 142, 3)
model = create_model(input_shape)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy','recall'])
# model.summary()

In [7]:


batch_size = 2
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

history = model.fit(
    [target_images, X2_train_images], [y1_train, y2_train],
    epochs=50,
    validation_split=0.2,
    # callbacks=[early_stopping]
)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 70s 70s/step - dense_3_accuracy: 0.0000e+00 - dense_4_recall: 0.0000e+00 - loss: 1.0000 - val_dense_3_accuracy: 0.0000e+00 - val_dense_4_recall: 0.0000e+00 - val_loss: 0.9822
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 65s 65s/step - dense_3_accuracy: 0.0000e+00 - dense_4_recall: 0.0000e+00 - loss: 0.9822 - val_dense_3_accuracy: 0.0000e+00 - val_dense_4_recall: 0.0000e+00 - val_loss: 0.9429
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 77s 77s/step - dense_3_accuracy: 0.0000e+00 - dense_4_recall: 0.0000e+00 - loss: 0.9429 - val_dense_3_accuracy: 0.0000e+00 - val_dense_4_recall: 0.0000e+00 - val_loss: 0.8640
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 58s 58s/step - dense_3_accuracy: 0.0000e+00 - dense_4_recall: 0.0000e+00 - loss: 0.8640 - val_dense_3_accuracy: 0.0000e+00 - val_dense_4_recall: 0.0000e+00 - val_loss: 0.7070
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 59s 59s/step - dense_3_accuracy: 0.0000e+00 - dense_4_recall: 0.0000e+00 - loss: 0.7070 - val_dense_3_accuracy: 0.0000e+

KeyboardInterrupt: 